In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf

import cst_model as cst
import distortion_layers as ly
import base_models as bm
import callbacks as cb


tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

### Create distortion layer

In [2]:
dcs = {
    "contrast": {"factor": [0.2, 0.2]},
    "color": {"factor": [20,0,20]},
    "blur": {"filter_shape": 2, "sigma": 5.},  # kernel size is 'filter_shape * 2 + 1'
    "brightness": {"lower": .85, "upper":1.15}
}

layers = [
    ly.RandomColorByChannel(**dcs["color"]), 
    tf.keras.layers.RandomContrast(**dcs["contrast"]),
    ly.RandomBrightness(**dcs["brightness"]),
    ly.RandomGaussianBlur(**dcs["blur"])
]


dist_layer = tf.keras.layers.Lambda(lambda x: x * 0.)

2022-06-27 20:38:35.255942: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 20:38:35.262724: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 20:38:35.263213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-27 20:38:35.264167: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Load images and assign parameters

In [3]:
train_path = "../data"
tile_size = 128
batch_size = 64
channels = 3
n_st_components = 2
alpha = 1
epochs = 10

gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2
)

t_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
    class_mode='binary',  # 'sparse' for multiclass, 'binary' for binary 
    subset='training'
)

v_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode="rgb",  # rgb for color
    batch_size=batch_size,
    shuffle=False,
    class_mode='binary',  # 'sparse' for multiclass, 'binary' for binary
    subset='validation'
)

Found 41770 images belonging to 2 classes.
Found 10442 images belonging to 2 classes.


### Train network

In [4]:
metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]

base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=1)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=True)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = metrics 
)

# cst_model.run_eagerly = True

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10


2022-06-27 20:30:05.097978: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


653/653 [==============================] - 278s 403ms/step - loss: 0.5440 - bce: 0.4334 - acc: 0.9462 - loss_0: 0.5875 - bce_: 0.5501 - acc_: 0.8381 - bce_0: 0.7127 - acc_0: 0.5598 - bce_1: 0.7127 - acc_1: 0.5598 - val_val_bce: 1.0674 - val_val_acc: 0.9074
Epoch 2/10
430/653 [==================>...........] - ETA: 1:26 - loss: 0.5201 - bce: 0.4035 - acc: 0.9778 - loss_0: 0.5197 - bce_: 3.2438 - acc_: 0.9036 - bce_0: 0.6904 - acc_0: 0.5777 - bce_1: 0.6904 - acc_1: 0.5777

KeyboardInterrupt: 

In [ ]:
cst_model.__dict__


In [ ]:
cst_model.compiled_loss._loss_metric.__dict__

In [ ]:
cst_model.compiled_metrics._metrics[0][0].__dict__

In [4]:
metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]

base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=1)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=alpha, n_st_components=n_st_components, 
                         dist_layers=dist_layer, binary=True)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = metrics 
)

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10


2022-06-27 20:38:58.566146: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


653/653 [==============================] - 279s 404ms/step - loss_0: 0.6021 - bce_: 0.5335 - acc_: 0.8052 - bce_0: 0.7319 - acc_0: 0.5601 - bce_1: 0.7319 - acc_1: 0.5601 - val_bce: 0.4705 - val_acc: 0.8889
Epoch 2/10
653/653 [==============================] - 261s 399ms/step - loss_0: 0.5213 - bce_: 0.4696 - acc_: 0.8879 - bce_0: 0.6985 - acc_0: 0.5734 - bce_1: 0.6985 - acc_1: 0.5734 - val_bce: 0.4656 - val_acc: 0.8931
Epoch 3/10
653/653 [==============================] - 260s 399ms/step - loss_0: 0.5144 - bce_: 0.4793 - acc_: 0.8932 - bce_0: 0.6926 - acc_0: 0.5772 - bce_1: 0.6926 - acc_1: 0.5772 - val_bce: 0.4916 - val_acc: 0.8957
Epoch 4/10
653/653 [==============================] - 260s 399ms/step - loss_0: 0.5122 - bce_: 0.5115 - acc_: 0.8959 - bce_0: 0.6898 - acc_0: 0.5789 - bce_1: 0.6898 - acc_1: 0.5789 - val_bce: 0.5311 - val_acc: 0.8975
Epoch 5/10
653/653 [==============================] - 260s 399ms/step - loss_0: 0.5121 - bce_: 0.5405 - acc_: 0.8978 - bce_0: 0.6881 - acc_0: 0